In [1]:
# Imports for plotting
import ipympl
%matplotlib widget
import matplotlib.pyplot as plt
# Has side effects allowing for 3D plots
from mpl_toolkits.mplot3d import Axes3D

In [2]:
from dolfin import *

# Create mesh and define function space
mesh = UnitSquareMesh(8, 8)
V = FunctionSpace(mesh, 'CG', 1)

# Define boundary conditions
u0 = Expression('1 + x[0]*x[0] + 2*x[1]*x[1]', degree=2)

def u0_boundary(x, on_boundary):
    return on_boundary

bc = DirichletBC(V, u0, u0_boundary)

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = Constant(-6.0)
a = inner(nabla_grad(u), nabla_grad(v))*dx
L = f*v*dx

# Compute solution
u = Function(V)
solve(a == L, u, bc)

In [3]:
uvals = u.compute_vertex_values(mesh)
xyvals = mesh.coordinates()
xvals = xyvals[:, 0]
yvals = xyvals[:, 1]

In [4]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.scatter(xvals, yvals, uvals)

FigureCanvasNbAgg()